<a href="https://colab.research.google.com/github/KuriaPatata/first-ai/blob/main/Image_Classification_(CIFAR_10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision and torchvision datasets
import torchvision
from torchvision import datasets

# Import data-specific utilities
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.data import sampler

# Import train-specific utilities
import torch.nn.functional as F
from torch import optim

# Improt matplotlib
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

# Improt sklearn.metrics
from sklearn.metrics import precision_score, f1_score, confusion_matrix

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

PyTorch version: 2.0.1+cu118
torchvision version: 0.15.2+cu118


In [ ]:
import urllib.request
import tarfile

# URL of the CIFAR-10 dataset
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
filename = "cifar-10-python.tar.gz"

# Download the dataset
urllib.request.urlretrieve(url, filename)

# Extract the dataset
with tarfile.open(filename, "r:gz") as tar:
    tar.extractall()

In [ ]:
# Set device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
# Define the transformations for the dataset
transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Download and load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define the data loaders for training and test sets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define the MLP architecture
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Initialize the models
mlp_model = MLP().to(device)
cnn_model = CNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Function to calculate precision and F1 score
def calculate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)
    return precision, f1

# Function to calculate confusion matrix
def calculate_confusion_matrix(y_true, y_pred):
    return confusion_matrix(y_true, y_pred)

In [ ]:
# Function to evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    precision, f1 = calculate_metrics(all_labels, all_preds)
    confusion_mat = calculate_confusion_matrix(all_labels, all_preds)
    return precision, f1, confusion_mat

In [ ]:
# Function to train the model
def train_model(model, data_loader, optimizer, criterion, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(data_loader):.4f}')

# Training the MLP model
num_epochs = 10
train_model(mlp_model, train_loader, mlp_optimizer, criterion, num_epochs)

# Evaluating the MLP model on the test set
mlp_precision, mlp_f1, mlp_confusion = evaluate_model(mlp_model, test_loader)

# Training the CNN model
num_epochs = 10
train_model(cnn_model, train_loader, cnn_optimizer, criterion, num_epochs)

# Evaluating the CNN model on the test set
cnn_precision, cnn_f1, cnn_confusion = evaluate_model(cnn_model, test_loader)

Epoch [1/10], Loss: 1.6801
Epoch [2/10], Loss: 1.4549
Epoch [3/10], Loss: 1.3356
Epoch [4/10], Loss: 1.2429
Epoch [5/10], Loss: 1.1585
Epoch [6/10], Loss: 1.0797
Epoch [7/10], Loss: 1.0038
Epoch [8/10], Loss: 0.9317
Epoch [9/10], Loss: 0.8652
Epoch [10/10], Loss: 0.7996
Epoch [1/10], Loss: 1.4608
Epoch [2/10], Loss: 1.0915
Epoch [3/10], Loss: 0.9139
Epoch [4/10], Loss: 0.7968
Epoch [5/10], Loss: 0.6993
Epoch [6/10], Loss: 0.6185
Epoch [7/10], Loss: 0.5476
Epoch [8/10], Loss: 0.4793
Epoch [9/10], Loss: 0.4156
Epoch [10/10], Loss: 0.3635


In [ ]:
# Printing the results for MLP
print('Results for MLP:')
print('Precision:')
print(mlp_precision)
print('F1-score:')
print(mlp_f1)
print('Confusion Matrix:')
print(mlp_confusion)

# Printing the results for CNN
print('Results for CNN:')
print('Precision:')
print(cnn_precision)
print('F1-score:')
print(cnn_f1)
print('Confusion Matrix:')
print(cnn_confusion)

# Comparing the two methods
print('Comparison of MLP and CNN:')
print('Precision:')
print(' MLP:', np.mean(mlp_precision))
print(' CNN:', np.mean(cnn_precision))
print('F1-score:')
print(' MLP:', np.mean(mlp_f1))
print(' CNN:', np.mean(cnn_f1))

Results for MLP:
Precision:
[0.63384615 0.6218721  0.45073375 0.33333333 0.4652568  0.52012384
 0.6023622  0.66018307 0.66108787 0.61356297]
F1-score:
[0.62582278 0.64550265 0.44012282 0.40806827 0.46362268 0.40826245
 0.60714286 0.61579509 0.64621677 0.59097978]
Confusion Matrix:
[[618  39  36  38  37   9  26  22 119  56]
 [ 29 671  17  32  17   7  18  14  65 130]
 [ 87  21 430 137 129  44  80  49  11  12]
 [ 29  26  76 526  49 123  89  38  16  28]
 [ 37  12 157 125 462  31  78  68  19  11]
 [ 19  17  72 356  65 336  55  49  13  18]
 [ 10  18  86 115  89  34 612  15   8  13]
 [ 37  17  48 123  96  40  15 577  18  29]
 [ 75  87  20  56  29  12  17  10 632  62]
 [ 34 171  12  70  20  10  26  32  55 570]]
Results for CNN:
Precision:
[0.77085533 0.84244373 0.61767729 0.5        0.68942731 0.58857143
 0.75848691 0.79034029 0.84935897 0.67663257]
F1-score:
[0.7498716  0.81324366 0.60922453 0.50980392 0.65618449 0.60292683
 0.77006401 0.75353218 0.82128099 0.75737561]
Confusion Matrix:
[[730

It is observed that in general both architectures (MLP, CNN) give good results in the classification of images both in precision and F1-score. However, CNN performs better on F1-score in more classes. Regarding the confusion matrix we observe that both architectures recognize the classes satisfactorily.